In [1]:
import numpy as np
import math
import random as rnd

import cv2

6. Система координат камеры получается из мировой поворотом на <code>45 градусов вокруг оси z и трансляцией на 10 вдоль оси z</code>. Внутренние параметры камеры: <code>fx=fy=400, cx=960, cy=540</code>. Написать программу, которая выводит на экран матрицу проекции камеры и целочисленные координаты пикселя на изображении, соответствующего трехмерной точке с координатами <code>(10, -10, 100)</code> в мировой системе координат.

In [2]:
r = np.around(math.cos(math.radians(45)),3)

rt = np.array([[math.cos(math.radians(45)), -math.sin(math.radians(45)), 0, 0],
              [math.sin(math.radians(45)), math.cos(math.radians(45)), 0, 0],
              [0, 0, 1, 10]], dtype=np.float64)

k = np.array([[400, 0, 960],
              [0, 400, 540],
              [0, 0, 1]], dtype = np.float64)

point = np.array([10, -10, 100, 1], dtype = np.float64)

p = k @ rt

n_point = p @ point

p = np.around(p, 2)

n_point = np.around(n_point)

print("Матрица проекции камеры:\n")
p


Матрица проекции камеры:



array([[ 2.8284e+02, -2.8284e+02,  9.6000e+02,  9.6000e+03],
       [ 2.8284e+02,  2.8284e+02,  5.4000e+02,  5.4000e+03],
       [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  1.0000e+01]])

In [3]:
print("Координаты пикселя: \n")

print("x = " + str(n_point[0]/n_point[2]))
print("y = " + str(n_point[1]/n_point[2]))

Координаты пикселя: 

x = 1011.4272727272727
y = 540.0


7. Используя прямое линейное преобразование (<code>DLT</code>) и <code>SVD</code>, написать программу, которая принимает на вход множество пар двумерных точек, и выводит на экран соответствующую этим парам матрицу гомографии. Написать генератор пар двумерных точек для теста алгоритма, удовлетворяющих заданному заранее преобразованию гомографии.

In [4]:
def pairs_generator(n):
    for i in range(n):
    	yield ([rnd.randint(0, 100), rnd.randint(0, 100), 1], [rnd.randint(0, 100), rnd.randint(0, 100), 1])

In [5]:
pairs = []

for i in pairs_generator(5):
    pairs.append(i)
    
pairs = np.array(pairs)

pairs[:, 0]

array([[ 9, 33,  1],
       [40,  2,  1],
       [15, 22,  1],
       [95, 74,  1],
       [76, 92,  1]])

In [6]:
U, S, Vt = np.linalg.svd(pairs[:, 0])
U, S, Vt

(array([[-0.16713394, -0.47440173, -0.56743693, -0.31197453, -0.57244778],
        [-0.17217176,  0.70937697, -0.51076357, -0.41121207,  0.19278595],
        [-0.14839334, -0.1478883 , -0.58120169,  0.6983537 ,  0.36140722],
        [-0.68188767,  0.33579711,  0.21802424,  0.36282357, -0.49304563],
        [-0.67485684, -0.37026568,  0.17834219, -0.33799067,  0.51130025]]),
 array([175.65994095,  37.02687817,   1.26310666]),
 array([[-0.72119701, -0.69265043, -0.01050008],
        [ 0.69267715, -0.72124638,  0.00142109],
        [ 0.00855746,  0.00624828, -0.99994386]]))

In [7]:
h_cv2, status = cv2.findHomography(pairs[:, 0], pairs[:, 1])
h_cv2

array([[  4.74892126,  -4.6005794 ,  16.5878798 ],
       [ 19.92599431, -17.32779177,  74.48005802],
       [  0.25356943,  -0.22564245,   1.        ]])

8. Модифицировать генератор из <code>Задачи 7</code> так, чтобы он выдавал <code>n=1000</code> соответствий, из которых <code>30%</code> соответствуют заданному заранее преобразованию гомографии, а остальные взяты случайно (преобразованию гомографии не удовлетворяют). Реализовать нахождение преобразования гомографии, используя <code>DLT</code> и <code>RANSAC</code>.